## 3.2.3 BM25를 활용 한 RAG System

In [ ]:
! pip install llama-index-retrievers-bm25
! pip install konlpy

### 1. 데이터 준비

In [1]:
### Step1. 데이터 로드
from llama_index.core import SimpleDirectoryReader
documents = SimpleDirectoryReader('./data').load_data()
print(f'문서의 총 개수 : {len(documents)}')

문서의 총 개수 : 23


In [2]:
from konlpy.tag import Okt
# Okt 형태소 분석기 초기화
okt = Okt()

# 문서 내용 토큰화 함수 정의
def tokenize_korean_text(text):
    return okt.morphs(text)  # 한국어 형태소 기반 토큰화
tokenize_korean_text('이 책은 RAG 시스템을 구축하는 방법에 대해 다룹니다.')

In [27]:
### Step 2. 청킹 및 인덱스 생성
# 청킹 (chunking)
from llama_index.core.node_parser import SentenceSplitter, TokenTextSplitter
# splitter = SentenceSplitter()
splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=20)
nodes = splitter.get_nodes_from_documents(documents)
print(f'노드의 개수 : {len(nodes)}')

노드의 개수 : 68


In [29]:
# BM25 기반 인덱스 생성(50페이지만 진행)
from llama_index.core.indices.keyword_table import KeywordTableIndex
index = KeywordTableIndex.from_documents(documents=documents,
                                         text_splitter=splitter,
                                         extract_keyword=tokenize_korean_text,
                                         max_keywords_per_node=10, # 각 청크에서 추출 된 상위 N개 키워드 선택
                                         max_nodes_per_query=10, # 쿼리에 대해 반환 할 노드 수를 제한
                                         show_progress=True # 진행 상황 확인
                                        )

Extracting keywords from nodes: 100%|██████████| 42/42 [00:40<00:00,  1.03it/s]


In [30]:
# 인덱스 로컬 경로 저장
index.storage_context.persist(persist_dir="./bm25_index_storage")

In [2]:
### Step6-2. 저장 된 인덱스 불러오기
from llama_index.core import StorageContext, load_index_from_storage
storage_context = StorageContext.from_defaults(persist_dir="./bm25_index_storage") 
index = load_index_from_storage(storage_context)

### 2. BM25 Retriever 객체 생성

In [8]:
from llama_index.retrievers.bm25 import BM25Retriever
bm25_retriever = BM25Retriever.from_defaults(
    index=index,          # BM25를 적용할 인덱스
    similarity_top_k=2,   # 유사도 기반 상위 k개 결과 반환

)
# from llama_index.retrievers.bm25 import BM25Retriever
# bm25_retriever = BM25Retriever.from_defaults(
#     nodes=nodes,
#     similarity_top_k=2,
# )

In [33]:
from llama_index.core.response.notebook_utils import display_source_node
# will retrieve context from specific companies
query = '변동금리'
retrieved_nodes = bm25_retriever.retrieve(query)
for node in retrieved_nodes:
    display_source_node(node, source_length=5000)

**Node ID:** 3ab5b7c5-4389-43ac-8466-34b9ce716d1c<br>**Similarity:** 1.7117327451705933<br>**Text:** 23
ㄱ 
4.0%, 9개월후 4.5%가 된다. 자금 차입자 입장에서는 앞으로 시장금리가 상승할 것으로 
예상하는 경우에는 고정금리를 이용하는 것이 유리하고 시장금리가 하락할 것으로 예상
하는 경우에는 변동금리를 이용하는 것이 유리하다. 주요 선진국 중앙은행의 정책금리 
인상, 양적완화 축소 등 글로벌 금융긴축으로의 전환이 우리나라에도 시장금리 상승을 
야기하고 있는 상황에서 주택담보대출 차입자의 재무건전성을 유지하기 위해 정부가 
고정금리대출을 장려하는 것도 이런 이유 때문이다. 일반적으로 동일 만기에서는 고정금
리가 변동금리 보다 높은데 이는 자금 대여자(은행)에게 약정 기간중 금리 변동에 따른 
위험 프리미엄이 존재하기 때문이다 . 
 연관검색어 : 변동금리, 양도성예금증서 (CD)
고정금리부채권(SB)
고정금리부채권(Straight Bond)이란 정해진 기일에 고정된 이자를 지급하고 정해진 
만기에 원금을 지급하는 가장 일반적인 형태의 채권으로 전환사채(CB; Convertible 
Bond)의 주식전환권과 같이 특별한 조건이 없는 채권을 말한다. 회사채, 국채 등 대부분
의 채권이 고정금리부로 발행된다. 이에 대비되는 채권으로 변동금리부채권 (FRN; 
Floating Rate Note)이 있다. 이는 정해진 기일에 특정 금리(예: 국내에서는 3개월 CD금
리, 해외에서는 LIBOR금리)에 연동된 금리를 지급하고 정해진 만기에 원금을 지급하는 
채권이다. 고정금리부채권에 대한 투자는 향후 금리 하락이 예상될 때 유리하며 금리 
상승이 예상될 때에는 변동금리부채권에 투자하는 것이 유리하다. 한편 인플레이션이 
심할 때 인플레이션을 헤지하기 위한 목적으로 발행되는 채권이 물가연동채권인데 이는 
원금이 물가상승률(주로 CPI 사용)에 연동하여 증가한다는 면에서 변동금리부채권과 
구별된다.<br>

**Node ID:** b6dd70a0-d98f-4b74-b49e-c95e47594fcb<br>**Similarity:** 0.0<br>**Text:** 한편 인플레이션이 
심할 때 인플레이션을 헤지하기 위한 목적으로 발행되는 채권이 물가연동채권인데 이는 
원금이 물가상승률(주로 CPI 사용)에 연동하여 증가한다는 면에서 변동금리부채권과 
구별된다.
 연관검색어 : 변동금리부채권 (FRN)
고정분류여신
금융기관은 정기적으로 차주의 채무상환능력과 금융거래 내용 등을 감안하여 보유자
산 의 건전성을 5단계(정상, 요주의, 고정, 회수의문, 추정손실)로 분류하여 각각 그에 
상응하는 적정 수준의 대손충당금을 적립하여야 한다. 고정분류여신은 차주 채무상환능
고정분류여신 ∙<br>

### 3. LLM 답변 생성

In [34]:
# API KEY 설정
from dotenv import load_dotenv
import os
load_dotenv()
os.environ['OPEANAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [36]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
Settings.llm = OpenAI(model="gpt-4o", temperature=0.5)

In [37]:
query_engine = index.as_query_engine()
response = query_engine.query(query)
print(response)

변동금리는 일정 주기별로 시장 금리를 반영하여 약정금리가 변동하는 금리를 의미합니다. 예를 들어, 만기가 1년이고 변동 주기가 3개월인 경우, 약정금리는 특정 기준 금리(예: CD금리)에 일정한 가산금리를 더하여 3개월마다 변동된 시장 금리에 연동하여 조정됩니다. 변동금리는 시장 금리가 하락할 것으로 예상될 때 유리합니다.
